In [ ]:
import numpy as np
from my_stuff import MufexKeys
from quantfreedom.custom_logger import CustomLogger
from quantfreedom.enums import CandleProcessingType
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.strategies.strategy import Strategy
from quantfreedom.base import backtest_df_only
from quantfreedom.enums import *
from quantfreedom.helper_funcs import create_os_cart_product_nb

mufex = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
)
symbol = "BTCUSDT"
%load_ext autoreload
%autoreload 2

In [ ]:
candles = mufex.get_candles_df(symbol=symbol, timeframe='5m', candles_to_dl=500)

In [ ]:
mufex.set_exchange_settings(
    symbol=symbol,
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings()
order_settings_arrays = OrderSettingsArrays(
    risk_account_pct_size=np.array([1.0]) / 100,
    sl_based_on_add_pct=np.array([0.01]) / 100,
    sl_based_on_lookback=np.array([50]),
    risk_reward=np.array([1.0]),
    leverage_type=np.array([LeverageStrategyType.Dynamic]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    increase_position_type=np.array([IncreasePositionType.SmalletEntrySizeAsset]),
    stop_loss_type=np.array([StopLossStrategyType.SLBasedOnCandleBody]),
    take_profit_type=np.array([TakeProfitStrategyType.Provided]),
    max_equity_risk_pct=np.array([4]) / 100,
    long_or_short=np.array([LongOrShortType.Long]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct_from_candle_body=np.array([0.0]) / 100,
    sl_to_be_zero_or_entry_type=np.array([SLToBeZeroOrEntryType.Nothing]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.High]),
    trail_sl_when_pct_from_candle_body=np.array([3.0]) / 100,
    trail_sl_by_pct=np.array([1.0]) / 100,
    static_leverage=np.array([0.0]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
    num_candles=np.array([150, 250]),
    entry_size_asset=np.array([0.0]),
    max_trades=np.array([3]),
)
os_cart_arrays = create_os_cart_product_nb(
    order_settings_arrays=order_settings_arrays,
)

In [ ]:
logger = CustomLogger(log_debug=True, disable_logging=False)
strategy = Strategy(
    candle_processing_mode=CandleProcessingType.Backtest,
    candles=candles.values,
    logger=logger,
)
candles_reg = backtest_df_only(
    starting_equity=1000.0,
    backtest_settings=backtest_settings,
    os_cart_arrays=os_cart_arrays,
    exchange_settings=mufex.exchange_settings,
    candles=candles,
    strategy=strategy,
    logger=logger,
)

In [ ]:
candles_reg.sort_values(['ind_set_idx','or_set_idx']).iloc[:]

In [ ]:
logger = CustomLogger(log_debug=True, disable_logging=False)

strategy = Strategy(
    candle_processing_mode=CandleProcessingType.RealBacktest,
    candles=candles.values,
    logger=logger,
)
candle_backtest = backtest_df_only(
    starting_equity=1000.0,
    backtest_settings=backtest_settings,
    os_cart_arrays=os_cart_arrays,
    exchange_settings=mufex.exchange_settings,
    candles=candles,
    strategy=strategy,
    logger=logger,
)

In [ ]:
candle_backtest.sort_values(['ind_set_idx','or_set_idx'])